Task:Restaurant recommendation

In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/project/Dataset .csv')
df.head()

In [48]:
df['Votes'].fillna(df['Votes'].mean(), inplace=True)
df['Cuisines'].fillna('Unknown', inplace=True)

<ipython-input-48-a03dadbd45b8>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Votes'].fillna(df['Votes'].mean(), inplace=True)
<ipython-input-48-a03dadbd45b8>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [16]:
encoder = OneHotEncoder(sparse_output=False)
encoded_cuisines = encoder.fit_transform(data[['Cuisines']])
encoded_cuisines_df = pd.DataFrame(encoded_cuisines, columns=encoder.get_feature_names_out(['Cuisines']))

In [19]:
df = pd.concat([data.reset_index(drop=True), encoded_cuisines_df], axis=1)

In [20]:
data['Aggregate rating'] = data['Aggregate rating'] / data['Aggregate rating'].max()
data['Votes'] = data['Votes'] / data['Votes'].max()

In [21]:
user_preferences = {
    'cuisines': ['Japanese', 'Korean'],
    'price_range': 3,  # Medium price range
    'rating_threshold': 0.8  # Threshold for normalized ratings
}

In [38]:
data['Restaurant Name'] = data.index.astype(str)

In [46]:
if 'Restaurant Name' not in data.columns:
    data['Restaurant Name'] = data.index.astype(str)  # Create one if missing

required_columns = ['Restaurant Name', 'Aggregate rating', 'Price range','Cuisines'] + list(encoded_cuisines_df.columns)
filtered_data = df[
    (data['Price range'] == user_preferences['price_range']) &
    (data['Aggregate rating'] >= user_preferences['rating_threshold'])
][required_columns]

In [30]:
print("Available columns in the dataset:", data.columns)


Available columns in the dataset: Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')


In [47]:
# Function to recommend restaurants
def recommend_restaurants(user_likes, filtered_data, similarity_matrix, top_n=5):
    indices = filtered_data[filtered_data['Restaurant Name'].isin(user_likes)].index
    scores = similarity_matrix[indices].mean(axis=0)  # Aggregate similarity
    top_indices = scores.argsort()[-top_n:][::-1]  # Get top matches

    recommendations = filtered_data.iloc[top_indices]
    return recommendations[['Restaurant Name', 'Cuisines', 'Aggregate rating', 'Price range']]

# Step 4: Test the recommendation system
sample_user_likes = ['Ooma']  # Example: User likes this restaurant
recommendations = recommend_restaurants(sample_user_likes, filtered_data, similarity_matrix)

print("Recommendations:")
print(recommendations)


Recommendations:
                       Restaurant Name                          Cuisines  \
9547                      Ceviz A��ac۱   World Cuisine, Patisserie, Cafe   
1259                    The Grill Mill  North Indian, Continental, Asian   
1158  Cafe Al Fresco by Cantina Bodega                              Cafe   
1164                            Zambar     South Indian, Seafood, Kerala   
1212                       Big Wong XL              Chinese, Thai, Sushi   

      Aggregate rating  Price range  
9547          0.857143            3  
1259          0.816327            3  
1158          0.877551            3  
1164          0.816327            3  
1212          0.816327            3  


<ipython-input-47-01d0bb9da37e>:4: RuntimeWarning: Mean of empty slice.
  scores = similarity_matrix[indices].mean(axis=0)  # Aggregate similarity
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
